<a href="https://colab.research.google.com/github/koh-ts/traffic_volume_prediction/blob/master/traffic_volume_prediction_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
import pandas as pd
import datetime
import pandas_profiling as pdp
from IPython.display import HTML
import xgboost as xgb
import lightgbm as lgb

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
root_dir = '/content/drive/My Drive/report1/'

N = 30000
nfeat = 8

train_data = pd.read_csv(root_dir + 'train.csv')
test_data = pd.read_csv(root_dir + 'test.csv')

Xmat = np.matrix(train_data)[:,:nfeat]
Ymat = np.matrix(train_data)[:,nfeat]

In [0]:
all_data = pd.concat([train_data, test_data], ignore_index=True)
all_data = all_data[['timestamp', 'holiday', 'temperature', 'weather', 'weather_detail', 'rain_in_hour', 'snow_in_hour', 'clouds_cover', 'traffic_volume']]
# all_data.head(4)

In [0]:
# profile1 = pdp.ProfileReport(all_data)
# profile1.to_file(outputfile="myoutputfile1.html")
# HTML(filename='myoutputfile1.html')

In [0]:
mean_traffic = [844.1971481711097, 520.6844720496895, 391.89875, 372.22609246358456, 701.162890385793, 2084.489069331668, 4167.179251074279, 4742.792975970425, 4608.443757725587, 4386.964470284238, 4189.753390875462, 4475.000661375661, 4729.360078277887, 4728.403106009453, 4949.089494163424, 5252.46875, 5675.80192926045, 5341.854208084825, 4283.009658725048, 3287.383257030739, 2841.853092783505, 2688.3029525032093, 2215.9378566899177, 1472.6247678018576]

In [0]:
# cnt = 0
for index, d in all_data.iterrows():
  # cnt += 1
  y = int(d['timestamp'][0:4])
  m = int(d['timestamp'][5:7])
  dd = int(d['timestamp'][8:10])
  n = int(d['timestamp'][11:13])
  if all_data.iat[index - n, 1] != 'None':
    all_data.at[index, 'isholiday'] = 1
  else:
    all_data.at[index, 'isholiday'] = 0
  all_data.at[index, 'time_traffic'] = mean_traffic[n]
  all_data.at[index, 'time'] = n
  dayy = datetime.date(y, m, dd).weekday()
  all_data.at[index, 'day'] = dayy
  # if index == len(train_data):
  #   print(d)
  # if cnt == 100:
  #   break

In [0]:
all_data['weather_detail'] = all_data['weather_detail'].str.replace('Sky is clear', 'clear')
all_data['weather_detail'] = all_data['weather_detail'].str.replace('sky is clear', 'clear')
all_data['weather_detail'] = all_data['weather_detail'].str.replace('Sky is Clear', 'clear')
all_data['temperature'] = all_data['temperature'].replace(0.0, all_data['temperature'].median())

In [0]:
# all_data[all_data['holiday'] == "Martin Luther King Jr Day"]

In [0]:
meta_t = pd.get_dummies(all_data['time'])
meta_w = pd.get_dummies(all_data['weather'])
meta_wd = pd.get_dummies(all_data['weather_detail'])

In [12]:
profile2 = pdp.ProfileReport(all_data)
profile2.to_file(outputfile="myoutputfile2.html")
HTML(filename='myoutputfile2.html')

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,13
Number of observations,45645
Total Missing (%),1.3%
Total size in memory,4.5 MiB
Average record size in memory,104.0 B
Numeric,8
Categorical,4
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [0]:
del all_data['timestamp']
del all_data['weather']
del all_data['holiday']
del all_data['weather_detail']

In [0]:
all_data = all_data[['temperature', 'rain_in_hour', 'snow_in_hour', 'clouds_cover', 'time', 'isholiday', 'day', 'time_traffic', 'traffic_volume']]
all_data.columns
l = len(all_data.columns) - 1
# all_data.head(3)

In [0]:
X = np.float64(np.hstack([meta_w.iloc[:len(train_data),:],meta_wd.iloc[:len(train_data),:],meta_t.iloc[:len(train_data),:],all_data.iloc[:len(train_data),:l]]))
Y = np.float64(Ymat)
XTrain, XVal, YTrain, YVal = train_test_split(X, Y, test_size = 0.2, random_state = 66)

In [0]:
def RMSE(vec1, vec2):
  return np.sqrt(np.mean(np.power(vec1-vec2,2)))

In [0]:
reg = Ridge(alpha=100).fit(XTrain, YTrain)

In [0]:
yHatTrain = reg.predict(XTrain)
yHatVal = reg.predict(XVal)
score_reg = cross_val_score(reg, X, Y, cv=5, n_jobs=-1)
print(score_reg)
print(score_reg.mean())

print("Training error ", RMSE(YTrain, yHatTrain))
print("Validation error ", RMSE(YVal, yHatVal))

In [0]:
dtrain = xgb.DMatrix(XTrain, label=YTrain)
dvalid = xgb.DMatrix(XVal, label=YVal)
num_round = 5000
evallist = [(dvalid, 'eval'), (dtrain, 'train')]
evals_result = {}
param = {'max_depth': 3, 'eta': 0.01, 'objective': 'reg:squarederror',}
bst = xgb.train(param, dtrain, num_round, evallist, evals_result=evals_result, early_stopping_rounds=1000)

In [0]:
XTest = np.float64(np.hstack([meta_w.iloc[len(train_data):,:],meta_wd.iloc[len(train_data):,:],meta_t.iloc[len(train_data):,:],all_data.iloc[len(train_data):,:l]]))

In [0]:
dtest = xgb.DMatrix(XTest)
yHatTest_xgb = bst.predict(dtest)
np.reshape(yHatTest_xgb, (len(yHatTest_xgb), 1))
np.savetxt(root_dir+'result_xgb.txt', yHatTest_xgb)

In [0]:
yHatTest = reg.predict(XTest)
np.savetxt(root_dir+'result.txt', yHatTest) #save predictions in rows

In [0]:
res = []
for i in YTrain:
  d = float(i[0])
  res.append(d)
resv = []
for i in YVal:
  d = float(i[0])
  resv.append(d)

In [0]:
train_data_set = lgb.Dataset(XTrain, res)
valid_data_set = lgb.Dataset(XVal, resv, reference=train_data_set)

params = {'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': {'rmse'},
          'num_leaves': 41,
          'learning_rate': 0.05,
          'feature_fraction': 0.9, #0.9
          'bagging_fraction': 0.7, #0.8
          'bagging_freq': 7,
          'lambda_l2': 2}

In [0]:
gbm = lgb.train(params, train_data_set, num_boost_round=300, valid_sets=[train_data_set, valid_data_set], early_stopping_rounds=10)
yHatTest_gbm = gbm.predict(XTest)
np.reshape(yHatTest_gbm, (len(yHatTest_gbm), 1))

In [0]:
np.savetxt(root_dir+'result_lgbm_last.txt', yHatTest_gbm)